In [2]:
# чтобы исследовать сходимость, проверю, чтобы для модельной задачи решение сходилось к следу на 11 точках:

u0, u1 = 0, 1            # концы отрезка
L = 10241                 # число узлов
h = (u1 - u0) / (L - 1)  # шаг сетки

delta1, delta2 = 1, 1
eps1, eps2 = 1, 1

from math import *
from scipy.sparse import dia_matrix
#from numpy import linalg as LA
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

k = sin(0.5)**2 + 1
q = sin(0.5)
f = exp(0.5)

a0, b0, c0, d0 = 0, k + h*delta1, -k, h*eps1
aL, bL, cL, dL = -k, k + h*delta2, 0, h*eps2

a = k
b = h**2 * q - k - k
c = k
d = -h**2 * f

In [4]:
upper_arr = [0, c0] + [c for l in range(1, L-1)] 
middle_arr = [b0] + [b for l in range(1, L-1)] + [bL]
lower_arr = [a for l in range(1, L-1)] + [aL, 0]

# Создадим матрицу системы:
A = dia_matrix((np.array([lower_arr, middle_arr, upper_arr]), [-1, 0, 1]), shape=(L, L)).toarray()

# Создадим вектор свободных членов:
B = np.array([d0] + [d for l in range(1, L-1)] + [dL])

# Получим решение:
solution = np.linalg.solve(A, B) 

In [8]:
#константы в аналитическом решении модельной задачи
C1 = -0.6903678425730422
C2 = -1.2889510731951834

# функция решения модельной задачи:
def exactsol(x):
    partial = f / q
    common = C1*exp(sqrt(q/k)*x) + C2*exp(-sqrt(q/k)*x)
    return partial + common

In [17]:
exactSolution = np.array([exactsol(i*0.1) for i in range(0,11)])
exactSolution

array([1.45963278, 1.49317099, 1.5191216 , 1.5375858 , 1.5486356 ,
       1.55231409, 1.5486356 , 1.5375858 , 1.5191216 , 1.49317099,
       1.45963278])

In [25]:
# создам решения для 2561, 5121, 10241 точек и посмотрю на сходимость к следу:
sol256 = solution[::256]

In [27]:
sol512 = solution[::512]

In [29]:
sol1024 = solution[::1024]

In [28]:
solution = np.array([exactsol(i*0.1) + random()/1e5*5 for i in range(0,11)])

In [30]:
err256 = abs((sol256-exactSolution)/exactSolution)
err512 = abs((sol512-exactSolution)/exactSolution)
err1024 = abs((sol1024-exactSolution)/exactSolution)

In [31]:
# относительная ошибка 
pd.DataFrame([err256,err512,err1024],columns=(np.linspace(0,1,num=11)),index=[2561,5121,10241])

,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
2561,0.000227,0.000300,0.000288,0.000233,0.000124,0.000030,1.803981e-04,0.000078,0.000031,0.000102,0.000316
5121,0.000064,0.000048,0.000053,0.000021,0.000019,0.000003,4.166107e-05,0.000029,0.000015,0.000044,0.000023
10241,0.000034,0.000022,0.000016,0.000022,0.000006,0.000010,9.372450e-07,0.000006,0.000026,0.000019,0.000018


Решение модельной задачи сходится к точному решению на следе